In [57]:
import requests
import json

Before starting, you'll need to create an account at [Writer](writer.com), then grab your API Bearer token and organization ID.

Create a file named `.env` in this directory with the lines 
```
WRITER_ORG_ID=YOUR_ORG_ID
WRITER_API_KEY=YOUR_API_KEY
```
or simply set the variables in the cell below.

In [58]:
from dotenv import load_dotenv
import os

load_dotenv()
org_id = os.environ.get("WRITER_ORG_ID")
api_key = os.environ.get("WRITER_API_KEY")

First let's request a list of models to see what's available and make sure your authorization is working:

In [59]:
url = f"https://enterprise-api.writer.com/llm/organization/{org_id}/model"

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": api_key
}

response = requests.get(url, headers=headers)

print(json.dumps(response.json(), indent=1))

{
 "models": [
  {
   "name": "Palmyra Base",
   "id": "palmyra-base",
   "type": "GPT"
  },
  {
   "name": "Palmyra Large",
   "id": "palmyra-large",
   "type": "GPT"
  },
  {
   "name": "Palmyra Instruct",
   "id": "palmyra-instruct",
   "type": "Instruct"
  },
  {
   "name": "PalmyraR",
   "id": "palmyra-r",
   "type": "GPT"
  }
 ]
}


Let's use `palmyra-instruct` for now:

In [60]:
model_id = "palmyra-instruct"
url = f"https://enterprise-api.writer.com/llm/organization/{org_id}/model/{model_id}/completions"


And send a basic request:

In [61]:
prompt = "why is the sky blue?"

response = requests.post(url, json={"prompt": prompt}, headers=headers)
print(json.dumps(response.json(), indent=1))

{
 "choices": [
  {
   "text": "\n\nThe sky is blue because of an effect called Rayleigh scattering. When",
   "logprobs": null
  }
 ]
}


If you run the previous code segment a few times, you'll see the explanation isn't the most detailed, and sometimes can't even finish a sentence. Let's increase the length of the response by adding a `maxTokens` parameter to our request:

In [62]:
payload = {
    "prompt": "why is the sky blue?",
    "maxTokens": 100
}

response = requests.post(url, json=payload, headers=headers)

# We can also strip the response down to just the main content to make it a bit cleaner
if response.status_code == 200:
    print(json.dumps(response.json()["choices"][0]["text"], indent=1))
else:
    print(response)

"\n\nThe sky is blue because of an effect called Rayleigh scattering, which is the scattering of light by particles in the atmosphere. When sunlight strikes the atmosphere, shorter-wavelength blue light is scattered in all directions more than other colors, giving the sky its blue color."


Now the answer is a bit more comprehensive. But maybe you don't want the response to change every time? Setting the temperature to 0 will make the response deterministic:

In [63]:
payload = {
    "prompt": "why is the sky blue?",
    "maxTokens": 100,
    "temperature": 0
}

response = requests.post(url, json=payload, headers=headers)

# We can also strip the response down to just the main content to make it a bit cleaner
if response.status_code == 200:
    print(json.dumps(response.json()["choices"][0]["text"], indent=1))
else:
    print(response)

"\n\nThe sky is blue because of the way sunlight interacts with the atmosphere. When sunlight reaches Earth's atmosphere, it is scattered in all directions by all the gases and particles in the air. Blue light is scattered more than other colors because it travels as shorter, smaller waves. This is why we see a blue sky most of the time."


No matter how many times you ask the question it will give the same answer in the same wording. 

For a full list of API endpoints and available parameters see [here](https://dev.writer.com/reference).

If you'd like to use the Writer Python library instead of requests, check the other examples in this repository.